<a href="https://colab.research.google.com/github/HarshR-03/MegaProject_Naman_Harsh_Sudhanshu/blob/main/Review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf

d = pd.read_csv("//content//drive//MyDrive//reviews.csv")
d["score"]-=1
d = d.dropna() #since text column has lots of nulls at the end



In [ ]:
d["score"].value_counts()

4.0    126387
3.0     29118
0.0     18296
2.0     15624
1.0     10575
Name: score, dtype: int64

In [ ]:
d["score"].value_counts()

5.0    126387
4.0     29118
1.0     18296
3.0     15624
2.0     10575
Name: score, dtype: int64

In [ ]:
import numpy as np
features  = d["text"][1:]
features.info()
target = np.asarray(d["score"][1:]).astype('int64')
tf.convert_to_tensor(target)
data = tf.data.Dataset.from_tensor_slices((features,target))

<class 'pandas.core.series.Series'>
RangeIndex: 199999 entries, 1 to 199999
Series name: text
Non-Null Count   Dtype 
--------------   ----- 
199999 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [ ]:
for f,t in data.take(5):
   print(f.numpy().decode("utf-8"))
   print("Label:", t.numpy())

Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
Label: 0
This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.
Label: 3
If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The flavor is very medicinal.
Label: 1
Great taffy at a great price.  There was a wide asso

In [ ]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [ ]:
train_set, valid_set, test_set = get_dataset_partitions_tf(data,199999)

In [ ]:
train_set = train_set.batch(32).prefetch(1)
valid_set = valid_set.batch(32).prefetch(1)
test_set = test_set.batch(32).prefetch(1)

In [ ]:
for f,t in train_set.take(5):
  print(f.numpy())
  print("label:",t)

[b'These are seriously delicious, especially if you like plain potato chips. My only concern is they\'re very oily. I understand they\'re "kettle chips" and they\'re "fried" (small batches and all) but when you open the bag (That\'s another thing, very strong bag! Need scissors!) there\'s a coating of oil on the inside of the bag and on the chips themselves. I suggest a blot/shake on some paper towels before consuming. Either way, they have a yummy potato taste, and very satisfying crunch. Delicious!'
 b"I found these at a super Gold Box pricing one day otherwise I would not have looked twice at 16.00 pancake mix.  Now we are hooked - nothing compares to this mix.  I've tried every box in the store for our belgian waffle maker and these top them all!<br />Waffles are nice and crisp on the outside and super soft on the inside. They are so dense they don't absorb syrup or toppings.  I will buy this over and over again."
 b'Here is a great way to get that tast of your childhood without al

In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size,mask_zero=True),
    tf.keras.layers.GRU(128,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(5,activation="softmax")
])
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)


Epoch 1/2
5000/5000 [==============================] - 125s 24ms/step - loss: 0.9401 - accuracy: 0.6584 - val_loss: 0.8712 - val_accuracy: 0.6702
Epoch 2/2
5000/5000 [==============================] - 98s 20ms/step - loss: 0.8501 - accuracy: 0.6808 - val_loss: 0.7929 - val_accuracy: 0.7013


In [ ]:
for f,g in test_set.take(1):
  t,k = f,g


In [ ]:
pred = model.predict(t)

1/1 [==============================] - 2s 2s/step


In [ ]:
pred

array([[1.08215220e-01, 2.02013761e-01, 3.88248414e-01, 2.16320023e-01,
        8.52026045e-02],
       [1.08840719e-01, 1.37087852e-01, 2.68199086e-01, 2.60273129e-01,
        2.25599229e-01],
       [4.07764548e-03, 9.32454411e-03, 4.90314662e-02, 3.71978223e-01,
        5.65588117e-01],
       [5.54229431e-02, 1.22881114e-01, 3.68146569e-01, 3.19195896e-01,
        1.34353489e-01],
       [4.29105945e-03, 2.02793698e-03, 5.84528921e-03, 6.83526024e-02,
        9.19483125e-01],
       [2.34362744e-02, 2.84988973e-02, 8.52771401e-02, 2.73566544e-01,
        5.89221179e-01],
       [4.21655597e-03, 1.03802215e-02, 6.24384172e-02, 4.01597917e-01,
        5.21366835e-01],
       [1.05133587e-02, 8.94129928e-03, 2.43934449e-02, 1.71405286e-01,
        7.84746587e-01],
       [7.85017014e-02, 2.55010854e-02, 3.28847654e-02, 1.01818390e-01,
        7.61294067e-01],
       [8.41865465e-02, 3.23756225e-02, 5.00746593e-02, 1.27526402e-01,
        7.05836833e-01],
       [6.78323135e-02, 2.3185

In [ ]:
k

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 3, 4, 3, 4, 3, 3, 4, 0, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 4, 0, 1,
       4, 4, 4, 3, 4, 0, 3, 4, 1, 2])>

In [ ]:
pred_arr = pred.argmax(axis=1)
pred_arr

array([2, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 2, 4, 4, 0, 0,
       4, 4, 4, 4, 4, 0, 3, 4, 0, 4])

In [ ]:
y = ["kill"]
y = np.asarray(y)
ypred = model.predict(y)
pred_arr = ypred.argmax(axis=1)
pred_arr

1/1 [==============================] - 0s 19ms/step


array([0])

In [ ]:
'''for item in f:
  print(f.shape, f.dtype)'''


x = ["terrible driver.",]
x_pred = np.array(x)
'''print(x_pred.shape)'''
h = model.predict(x_pred)
print(h)

1/1 [==============================] - 0s 321ms/step
[[-1.2078137]]


In [ ]:
#import 3rd party libraries
import requests
response = requests.get("https//www.xyz.com")

In [ ]:
import matplotlib as p
import matplotlib.pyplot as plt
import numpy as np
fig,ax=plt.subplots()
ax.scatter([1,2,3,4],[3,2,4,1])

